In [103]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LSTM, Dense

In [123]:
# Encoder
encoder_timesteps = 27
decoder_timesteps = 6
latent_dim = 16
encoder_in = Input(shape=(None, encoder_timesteps), name='encoder_in')
_, state_h, state_c = LSTM(units=latent_dim, return_state=True, name='encoder_lstm')(encoder_in)
encoder_states = [state_h, state_c]

In [124]:
# Decoder
decoder_in = Input(shape=(None, decoder_timesteps), name='decoder_in')
a, _, _ = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')(decoder_in, initial_state=encoder_states)
decoder_out = Dense(decoder_timesteps, activation='softmax')(a)

In [125]:
# Entire model
model = Model([encoder_in, decoder_in], decoder_out)
model.summary()

Model: "functional_52"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_in (InputLayer)         [(None, None, 27)]   0                                            
__________________________________________________________________________________________________
decoder_in (InputLayer)         [(None, None, 6)]    0                                            
__________________________________________________________________________________________________
encoder_lstm (LSTM)             [(None, 16), (None,  2816        encoder_in[0][0]                 
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 16), ( 1472        decoder_in[0][0]                 
                                                                 encoder_lstm[0][1]   

In [122]:
x = np.load('x.npy')
y = np.load('y.npy')
x = np.expand_dims(x, axis=2)
y = np.expand_dims(y, axis=2)
print(x.shape, y.shape)

(896500, 27, 1) (896500, 6, 1)


In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)